## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.
1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).
2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  
3. To generate vectors and split the instances into training and testing datasets at random.
4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.
5. To evaluate the performance of the classifier.

You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [1]:
import a2
from sklearn.svm import LinearSVC

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/guskhojra@GU.GU.SE/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/guskhojra@GU.GU.SE/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
gmbfile = open('/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [3]:
inputdata = a2.preprocess(gmbfile)
gmbfile.close()
inputdata[20:40]

['39\t2.0\tslogan\tNNS\tO\n',
 '41\t2.0\t""""\t``\tO\n',
 '42\t2.0\tbush\tNNP\tB-per\n',
 '43\t2.0\tnumber\tNN\tO\n',
 '45\t2.0\tterrorist\tNN\tO\n',
 '46\t2.0\t""""\t``\tO\n',
 '48\t2.0\t""""\t``\tO\n',
 '49\t2.0\tstop\tVB\tO\n',
 '51\t2.0\tbombings\tNNS\tO\n',
 '53\t2.0\t""""\t``\tO\n',
 '55\t3.0\tmarched\tVBD\tO\n',
 '58\t3.0\thouses\tNNS\tO\n',
 '60\t3.0\tparliament\tNN\tO\n',
 '63\t3.0\trally\tNN\tO\n',
 '65\t3.0\thyde\tNNP\tB-geo\n',
 '66\t3.0\tpark\tNNP\tI-geo\n',
 '68\t4.0\tpolice\tNNS\tO\n',
 '69\t4.0\tput\tVBD\tO\n',
 '71\t4.0\tnumber\tNN\tO\n',
 '73\t4.0\tmarcher\tNNS\tO\n']

## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [4]:
instances = a2.create_instances(inputdata)

In [5]:
instances[20:40]

[Class: gpe
  Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', 'week', 'restarted', 'part', 'conversion', 'process'],
 Class: geo
  Features: ['week', 'restarted', 'part', 'conversion', 'process', 'nuclear', 'plant', '<E1>', '<E2>', '<E3>'],
 Class: gpe
  Features: ['<S1>', '<S2>', '<S3>', '<S4>', 'official', 'say', 'expect', 'get', 'access', 'sealed'],
 Class: tim
  Features: ['access', 'sealed', 'sensitive', 'part', 'plant', 'surveillance', 'system', 'begin', 'functioning', '<E1>'],
 Class: org
  Features: ['sealed', 'sensitive', 'part', 'plant', 'surveillance', 'system', 'begin', 'functioning', '<E1>', '<E2>'],
 Class: org
  Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', 'backing', 'threatened', 'refer'],
 Class: gpe
  Features: ['<S1>', '<S2>', '<S3>', 'backing', 'threatened', 'refer'],
 Class: gpe
  Features: ['backing', 'threatened', 'refer', 'could', 'impose', 'sanction'],
 Class: org
  Features: ['backing', 'threatened', 'refer', 'could', 'impose', 'sanction', 'find', 'vi

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [6]:
bigdf = a2.create_table(instances)
bigdf[20:40]

,Class,0,1,2,3,4,5,6,7,8,...,990,991,992,993,994,995,996,997,998,999
20,gpe,38.047986,-27.560112,-9.860604,6.174779,-3.361988,-1.689744,5.972468,-0.040862,-2.407882,...,-0.021004,0.042738,0.006540,-0.015959,0.005428,0.022872,0.017623,0.006921,-0.008394,0.035247
21,geo,24.469999,34.376510,-0.682425,-0.166491,-3.260791,0.419476,0.198541,-8.346234,0.632233,...,-0.016794,-0.038653,-0.007228,0.035499,-0.017354,-0.009790,-0.028867,0.010713,0.012813,-0.000753
22,gpe,38.697205,-27.883748,-7.445610,3.838951,-1.293826,14.032032,1.562860,0.485280,0.796263,...,0.115145,-0.034346,0.053672,0.042628,-0.004813,-0.046035,0.054814,-0.103803,0.013482,-0.016702
23,tim,11.730680,15.397533,7.726629,0.879042,-15.203331,0.366493,-0.072721,10.648310,-1.425671,...,-0.019380,-0.004917,-0.017458,-0.001223,0.029066,-0.001231,-0.008188,0.002564,-0.011416,-0.008425
24,org,19.659739,26.917305,5.341885,0.312868,-13.598667,0.859161,0.029493,-6.184676,0.066454,...,-0.050457,0.000549,-0.014307,-0.040137,-0.006986,0.024804,0.018999,0.056630,-0.010243,0.002917
25,org,38.022851,-27.567501,-9.902211,6.180201,-3.337368,-1.751536,5.944718,-0.045653,-2.424820,...,-0.048818,0.009816,0.020476,-0.028995,-0.020027,-0.027242,-0.044190,0.055749,0.017396,-0.044952
26,gpe,34.906149,-25.039604,-4.030267,-1.500139,-0.656120,-3.802635,-5.049977,-0.816970,-1.107967,...,-0.046745,0.007509,0.015669,-0.028504,-0.033103,-0.034737,-0.046563,0.052380,0.016639,-0.058038
27,gpe,0.007148,0.000673,0.013597,-0.004198,-0.014816,-0.006780,0.008592,0.017571,0.002507,...,0.103571,-0.008962,-0.006045,0.098866,0.042479,0.104969,0.091481,-0.090978,-0.025796,0.122209
28,org,0.008020,0.001304,0.015617,-0.005082,-0.020303,-0.006453,0.006787,0.016223,0.010420,...,-0.054814,-0.008256,0.001896,-0.024401,-0.021134,-0.047071,-0.027126,0.031104,-0.010610,-0.019347
29,gpe,11.720036,15.392076,7.719531,0.860212,-15.194270,0.329529,-0.078205,10.657795,-1.425229,...,0.037246,-0.013719,-0.034466,-0.013871,0.017408,-0.006911,0.017214,-0.016106,-0.004257,0.006746


In [7]:
train_X, train_y, test_X, test_y = a2.ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

(array([[ 4.04132529e+01, -2.80614591e+01,  4.38304864e+00, ...,
         -2.39475928e-02, -2.15401192e-03,  2.32013651e-02],
        [ 1.17154754e+01,  1.53916479e+01,  7.71787709e+00, ...,
         -2.81012507e-02, -5.89439736e-03, -3.16625973e-02],
        [ 1.19031800e-01,  7.48333794e-03,  1.43098716e-01, ...,
          3.51708637e-02,  2.01753352e-02,  4.23200326e-02],
        ...,
        [ 3.54038047e+01, -2.52971674e+01, -3.37753826e+00, ...,
         -4.30019185e-03,  4.42664012e-03, -2.65378964e-02],
        [ 8.88141796e-02,  8.90738713e-03,  1.13824456e-01, ...,
          2.04503984e-02, -3.35149461e-03,  2.34824932e-03],
        [ 6.32475493e+01,  1.26403866e+01,  2.46842486e+00, ...,
          7.43310560e-03, -2.05235819e-02,  8.08372798e-03]]),
 array(['geo', 'geo', 'org', ..., 'tim', 'geo', 'per'], dtype=object),
 array([[ 2.86388926e+01, -2.01838295e+01,  4.26662661e+00, ...,
         -1.11251328e-02,  5.11345677e-02,  1.03087560e-02],
        [ 2.55608122e+01,  3.367

In [8]:
len(test_y) / (len(test_y) + len(train_y))

0.2

In [9]:
len(test_X) / (len(test_X) + len(train_X))

0.2

In [10]:
test_y[0]

'gpe'

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [11]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

/usr/local/lib64/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [12]:
train_predictions

array(['gpe', 'geo', 'org', ..., 'gpe', 'geo', 'gpe'], dtype=object)

In [13]:
train_y

array(['geo', 'geo', 'org', ..., 'tim', 'geo', 'per'], dtype=object)

In [14]:
test_predictions

array(['gpe', 'geo', 'per', 'geo', 'tim', 'gpe', 'org', 'gpe', 'gpe',
       'tim', 'geo', 'geo', 'geo', 'per', 'tim', 'geo', 'geo', 'gpe',
       'org', 'tim', 'tim', 'per', 'gpe', 'org', 'geo', 'geo', 'geo',
       'gpe', 'tim', 'per', 'gpe', 'org', 'geo', 'art', 'org', 'geo',
       'art', 'gpe', 'gpe', 'org', 'per', 'geo', 'gpe', 'gpe', 'geo',
       'gpe', 'geo', 'geo', 'org', 'geo', 'geo', 'gpe', 'org', 'gpe',
       'tim', 'tim', 'per', 'tim', 'gpe', 'gpe', 'geo', 'gpe', 'org',
       'tim', 'gpe', 'org', 'gpe', 'gpe', 'per', 'gpe', 'org', 'geo',
       'per', 'gpe', 'gpe', 'per', 'gpe', 'geo', 'gpe', 'gpe', 'gpe',
       'tim', 'tim', 'tim', 'geo', 'tim', 'org', 'org', 'gpe', 'gpe',
       'org', 'gpe', 'org', 'org', 'per', 'tim', 'org', 'per', 'geo',
       'geo', 'tim', 'gpe', 'per', 'org', 'gpe', 'tim', 'geo', 'tim',
       'gpe', 'geo', 'eve', 'org', 'gpe', 'tim', 'gpe', 'geo', 'geo',
       'tim', 'geo', 'per', 'tim', 'gpe', 'per', 'geo', 'org', 'geo',
       'tim', 'geo',

In [15]:
test_y

array(['gpe', 'org', 'geo', 'geo', 'per', 'per', 'org', 'gpe', 'gpe',
       'geo', 'gpe', 'geo', 'geo', 'eve', 'geo', 'tim', 'org', 'geo',
       'tim', 'tim', 'geo', 'per', 'geo', 'org', 'gpe', 'per', 'geo',
       'gpe', 'tim', 'per', 'gpe', 'gpe', 'geo', 'per', 'gpe', 'geo',
       'org', 'org', 'org', 'per', 'gpe', 'gpe', 'geo', 'tim', 'org',
       'art', 'geo', 'geo', 'geo', 'geo', 'geo', 'org', 'gpe', 'tim',
       'tim', 'tim', 'tim', 'per', 'gpe', 'gpe', 'per', 'tim', 'gpe',
       'org', 'tim', 'tim', 'gpe', 'per', 'geo', 'gpe', 'per', 'geo',
       'org', 'tim', 'per', 'org', 'geo', 'geo', 'gpe', 'tim', 'gpe',
       'tim', 'org', 'tim', 'org', 'gpe', 'org', 'gpe', 'org', 'geo',
       'geo', 'eve', 'org', 'org', 'per', 'org', 'gpe', 'org', 'org',
       'per', 'gpe', 'geo', 'per', 'tim', 'per', 'tim', 'gpe', 'gpe',
       'org', 'geo', 'tim', 'gpe', 'geo', 'gpe', 'gpe', 'gpe', 'geo',
       'tim', 'geo', 'geo', 'tim', 'org', 'gpe', 'gpe', 'geo', 'tim',
       'tim', 'geo',

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [16]:
a2.confusion_matrix(test_y, test_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,0.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0
eve,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
geo,2.0,0.0,48.0,24.0,0.0,11.0,8.0,19.0
gpe,2.0,1.0,17.0,39.0,0.0,18.0,9.0,11.0
nat,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
org,1.0,0.0,19.0,20.0,0.0,17.0,8.0,10.0
per,2.0,0.0,11.0,15.0,0.0,6.0,12.0,6.0
tim,1.0,2.0,9.0,16.0,0.0,10.0,8.0,14.0


In [17]:
a2.confusion_matrix(train_y, train_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,29.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
eve,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0
geo,0.0,0.0,358.0,26.0,0.0,9.0,1.0,3.0
gpe,0.0,0.0,2.0,397.0,0.0,5.0,5.0,0.0
nat,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0
org,0.0,0.0,3.0,13.0,1.0,269.0,8.0,0.0
per,0.0,0.0,6.0,25.0,0.0,6.0,191.0,4.0
tim,0.0,0.0,16.0,17.0,0.0,4.0,3.0,204.0


Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [18]:
train, test = a2.bonusb('/scratch/lt2222-v21-resources/GMB_dataset.txt')

/usr/local/lib64/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [19]:
test

,art,eve,geo,gpe,nat,org,per,tim
art,1.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0
eve,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0
geo,1.0,1.0,38.0,16.0,0.0,19.0,15.0,18.0
gpe,2.0,0.0,15.0,24.0,0.0,20.0,19.0,19.0
nat,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
org,4.0,1.0,9.0,9.0,0.0,20.0,19.0,6.0
per,1.0,1.0,9.0,6.0,0.0,18.0,16.0,10.0
tim,2.0,0.0,10.0,9.0,0.0,13.0,5.0,16.0


In [21]:
train

,art,eve,geo,gpe,nat,org,per,tim
art,27.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
eve,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0
geo,0.0,0.0,370.0,3.0,0.0,7.0,7.0,14.0
gpe,0.0,0.0,11.0,305.0,0.0,48.0,32.0,11.0
nat,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
org,0.0,0.0,3.0,0.0,0.0,282.0,9.0,7.0
per,2.0,0.0,1.0,5.0,0.0,11.0,198.0,6.0
tim,0.0,0.0,4.0,2.0,0.0,6.0,4.0,233.0
